Tools are pre-defined functions that agents can use. Agents can call tools to perform actions, such as searching the web, performing calculations, reading files, or calling remote APIs.

In [1]:
! pip install -q "autogen-agentchat==0.2.39" "groq==0.13.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 382.2/382.2 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.5 MB/s eta 0:00:00


In [2]:
import os
import autogen
from autogen import AssistantAgent, ConversableAgent
from google.colab import userdata

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
api_key = userdata.get('GROQ_API_KEY')
model = "llama-3.3-70b-versatile"

In [4]:
llm_config = {
    "model": model,
    "api_key": api_key,
    "api_type": "groq",
    }

In [5]:
from typing import Annotated, Literal
import os
from autogen import ConversableAgent

In [6]:
# Literal is used to specify that the Operator type
Operator = Literal["+", "-", "*", "/"]


# The Annotated type provides additional metadata. Metadata can be used by external tools for validation
def calculator(a: int, b: int, operator: Annotated[Operator, "operator"]) -> int:
    if operator == "+":
        return a + b
    elif operator == "-":
        return a - b
    elif operator == "*":
        return a * b
    elif operator == "/":
        return int(a / b)
    else:
        raise ValueError("Invalid operator")

In [7]:
assistant = ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant. "
    "You can help with simple calculations. "
    "Return 'TERMINATE' when the task is done.",
    llm_config=llm_config,
)

# User proxy agent is used for interacting with the assistant agent and executes tool calls.
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

In [8]:
from autogen import register_function

# Register the calculator function to the two agents.
register_function(
    calculator,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    name="calculator",  # By default, the function name is used as the tool name.
    description="A simple calculator",
)

In [9]:
chat_result = user_proxy.initiate_chat(assistant, message="What is (44232 + 13312 / (232 - 32)) * 5?")

User (to Assistant):

What is (44232 + 13312 / (232 - 32)) * 5?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_ncwx): calculator *****
Arguments: 
{"a": 232, "b": 32, "operator": "-"}
*******************************************************
***** Suggested tool call (call_as2c): calculator *****
Arguments: 
{"a": 13312, "b": 200, "operator": "/"}
*******************************************************
***** Suggested tool call (call_7vej): calculator *****
Arguments: 
{"a": 44232, "b": 66, "operator": "+"}
*******************************************************
***** Suggested tool call (call_qs9f): calculator *****
Arguments: 
{"a": 44298, "b": 5, "operator": "*"}
*******************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION calculator...

>>>>>>>> EXECUTING FUNCTIO

/usr/local/lib/python3.10/dist-packages/autogen/oai/groq.py:280: UserWarning: Cost calculation not available for model llama-3.3-70b-versatile
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)


Assistant (to User):

The final answer is 221490. 

TERMINATE

--------------------------------------------------------------------------------




*  `user_proxy` agent execute the function calling --> return int
*  `assistant` agent takes the arguments and suggests tools using LLM

